In [ ]:
# need to have mt5 app installed on your machine
# documentation of the package: https://www.mql5.com/en/docs/python_metatrader5

In [1]:
import MetaTrader5 as mt5
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [11]:
# data folder
# note: make sure this directory exists
DATA_FOLDER ="./data"

# broker credentials
ACCOUNT_NUM = <your-accnt-number-as-integer> # note: dont use string 
SERVER = <your-broker-server-name>
PASSWORD = <your-broker-password>
# investor = pock5wwb


# tick 
SYMBOL = "EURUSD"

# timeframes
START_YEAR = "2021"
END_YEAR = "2022"
# RESOLUTION = mt5.TIMEFRAME_MN1 # month
RESOLUTION = mt5.TIMEFRAME_M30 # 30 min


In [6]:
# broker timezone
utc_timezone = pytz.timezone("Etc/UTC")

In [7]:
# assume that it s utc
start_dt = datetime.strptime(START_YEAR, '%Y').replace(tzinfo=utc_timezone)
end_dt = datetime.strptime(END_YEAR, '%Y').replace(tzinfo=utc_timezone)

print(start_dt)
print(end_dt)

2021-01-01 00:00:00+00:00
2022-01-01 00:00:00+00:00


In [18]:
# intit mt5
if not mt5.initialize(login=ACCOUNT_NUM,
                      password=PASSWORD,
                      server=SERVER,
                      portable=True):
        print("error initializing")
        print(mt5.last_error())
        mt5.shutdown()


In [21]:
# login mt5
authorized = mt5.login(ACCOUNT_NUM, password=PASSWORD, server=SERVER)
if authorized:
    print("Successfully Authorized")
else:
    print("Not Authorized")
    print(mt5.last_error())


Successfully Authorized


In [30]:
# download for years

year_len = relativedelta(years=1)
start_ = start_dt
all_bars = pd.DataFrame()

while start_.year < end_dt.year:
        end_ = start_ + year_len - relativedelta(seconds=1)

        bars = mt5.copy_rates_range(SYMBOL, RESOLUTION, start_, end_)
        bars_df = pd.DataFrame(bars)
        bars_df.loc[:, ("time")] =  pd.to_datetime(bars_df['time'], unit='s')
        
        all_bars = pd.concat([all_bars, bars_df])
        
        start_ = start_ + year_len

In [31]:
all_bars.head(10)

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-01-04 00:00:00,1.22395,1.22396,1.22280,1.22372,249,15,0
1,2021-01-04 00:30:00,1.22365,1.22398,1.22327,1.22341,272,0,0
2,2021-01-04 01:00:00,1.22336,1.22424,1.22304,1.22399,1396,0,0
3,2021-01-04 01:30:00,1.22397,1.22528,1.22395,1.22501,1656,0,0
4,2021-01-04 02:00:00,1.22501,1.22501,1.22406,1.22455,2180,0,0
5,2021-01-04 02:30:00,1.22454,1.22457,1.22386,1.22407,1354,0,0
6,2021-01-04 03:00:00,1.22407,1.22503,1.22398,1.22445,1607,0,0
7,2021-01-04 03:30:00,1.22446,1.22557,1.22414,1.22526,1957,0,0
8,2021-01-04 04:00:00,1.22526,1.22584,1.22508,1.22548,2084,0,0
9,2021-01-04 04:30:00,1.22551,1.22585,1.22495,1.22538,1264,0,0


In [29]:
all_bars.describe()

,Open,High,Low,Close
count,12460.000000,12460.000000,12460.000000,12460.000000
mean,1.182802,1.183205,1.182388,1.182798
std,0.028206,0.028218,0.028195,0.028206
min,1.119090,1.119530,1.118610,1.119090
25%,1.164743,1.165170,1.164280,1.164725
50%,1.184910,1.185320,1.184590,1.184915
75%,1.207752,1.208143,1.207340,1.207742
max,1.234690,1.234940,1.233590,1.234690


In [27]:
bars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12460 entries, 0 to 12459
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         12460 non-null  datetime64[ns]
 1   open         12460 non-null  float64       
 2   high         12460 non-null  float64       
 3   low          12460 non-null  float64       
 4   close        12460 non-null  float64       
 5   tick_volume  12460 non-null  uint64        
 6   spread       12460 non-null  int32         
 7   real_volume  12460 non-null  uint64        
dtypes: datetime64[ns](1), float64(4), int32(1), uint64(2)
memory usage: 730.2 KB


In [32]:
# index and column renaming
all_bars = all_bars.set_index("time").sort_index()
all_bars.index.name = "Timestamp"

all_bars = all_bars[["open", "high", "low", "close"]]
all_bars = all_bars.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close"})
all_bars["Symbol"] = SYMBOL


all_bars.head(5)


,Open,High,Low,Close,Symbol
Timestamp,,,,,
2021-01-04 00:00:00,1.22395,1.22396,1.22280,1.22372,EURUSD
2021-01-04 00:30:00,1.22365,1.22398,1.22327,1.22341,EURUSD
2021-01-04 01:00:00,1.22336,1.22424,1.22304,1.22399,EURUSD
2021-01-04 01:30:00,1.22397,1.22528,1.22395,1.22501,EURUSD
2021-01-04 02:00:00,1.22501,1.22501,1.22406,1.22455,EURUSD


In [33]:
# save to file
all_bars.to_csv(f"{DATA_FOLDER}/mt5_{SYMBOL}.csv")